# Vector to raster tool collection

The following script helps to produce raster files from vector (polygon, lines, points) to raster files




read the lib

In [3]:
# geometry:
import geopandas as gpd
import re
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.wkt import loads
import geodatasets
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from rasterio import Affine

In [ ]:
INFO:




# (1) Vector-polygons to raster using rasterio

In [10]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

# Load the vector data (geopackage or shapefile)


################################################################## INPUT

input_vector_polygon = "N:/C2205_FAIRiCUBE/f02_data/tmp/test_vector_raster/vienna_polygon.shp"
resolution = 100 # pixel size in m
output_raster_tif = "N:/C2205_FAIRiCUBE/f02_data/tmp/test_vector_raster/vienna_polygon_100m_epsg3035_"+str(resolution)+".tif"#

attribute_name = 'BEZNR'

##################################################################


vector_file = input_vector_polygon
df = gpd.read_file(vector_file)

# Define the desired output raster shape (e.g., 1000x1000 pixels)
shape = 1000, 1000

# Generate an affine transform from the bounds of the vector data
transform = from_bounds(*df['geometry'].total_bounds, *shape)

# Rasterize the vector data
rasterize_rivernet = rasterize(
    #[(shape, 1) for shape in df['geometry']],
    [(geom, value) for geom, value in zip(df['geometry'], df[attribute_name])],
    out_shape=shape,
    transform=transform,
    fill=0,
    all_touched=True,
    dtype=rasterio.uint8
)

# Save the rasterized result as a GeoTIFF
with rasterio.open(
    output_raster_tif,
    'w',
    driver='GTiff',
    dtype=rasterio.uint8,
    count=1,
    width=shape[0],
    height=shape[1],
    transform=transform
) as dst:
    dst.write(rasterize_rivernet, indexes=1)

print("Vector data successfully converted to raster (GeoTIFF) without a template raster.")


Vector data successfully converted to raster (GeoTIFF) without a template raster.


# TESTING

https://gist.github.com/perrygeo/721040f8545272832a42
The current GDAL algorithms allow for two methods, both binary: the default centroid method and the all-touched method.

This is a third alternative which provides the percentage of coverage of each cell from 0 to 100 which can be thought of as pixel weights for many spatial analyses.

In [16]:
#https://gist.github.com/perrygeo/721040f8545272832a42

import rasterio
import fiona
import numpy as np
from rasterio import features
from affine import Affine
from shapely.geometry import shape, box



################################################################## INPUT

input_vector_polygon = "N:/C2205_FAIRiCUBE/f02_data/tmp/test_vector_raster/vienna_polygon.shp"
resolution = 100 # pixel size in m
output_raster_tif = "N:/C2205_FAIRiCUBE/f02_data/tmp/test_vector_raster/vienna_polygon_100m_epsg3035_"+str(resolution)+"percent.tif"#

attribute_name = 'BEZNR'

##################################################################


def _rasterize_geom(geom, shape, affinetrans, all_touched):
    indata = [(geom, 1)]
    rv_array = features.rasterize(
        indata,
        out_shape=shape,
        transform=affinetrans,
        fill=0,
        all_touched=all_touched)
    return rv_array


def rasterize_pctcover(geom, atrans, shape):
    alltouched = _rasterize_geom(geom, shape, atrans, all_touched=True)
    exterior = _rasterize_geom(geom.exterior, shape, atrans, all_touched=True)

    # Create percent cover grid as the difference between them
    # at this point all cells are known 100% coverage,
    # we'll update this array for exterior points
    pctcover = (alltouched - exterior) * 100

    # loop through indicies of all exterior cells
    for r, c in zip(*np.where(exterior == 1)):

        # Find cell bounds, from rasterio DatasetReader.window_bounds
        window = ((r, r+1), (c, c+1))
        ((row_min, row_max), (col_min, col_max)) = window
        x_min, y_min = (col_min, row_max) * atrans
        x_max, y_max = (col_max, row_min) * atrans
        bounds = (x_min, y_min, x_max, y_max)

        # Construct shapely geometry of cell
        cell = box(*bounds)

        # Intersect with original shape
        cell_overlap = cell.intersection(geom)

        # update pctcover with percentage based on area proportion
        coverage = cell_overlap.area / cell.area
        pctcover[r, c] = int(coverage * 100)

    return pctcover


if __name__ == "__main__":
    with fiona.open(input_vector_polygon) as src:
        geom = shape(next(src)['geometry'])  # take first feature's shapely geometry

    scale = 100
    atrans = Affine(1.0 / scale, 0.0, 0.0, 0.0, -1.0 / scale, 5.0)
    shape = (5 * scale, 5 * scale)
    profile = {
        'affine': atrans,
        'height': shape[0],
        'width': shape[1],
        'count': 1,
        'crs': {},
        'driver': 'GTiff',
        'dtype': 'uint8',
        'nodata': None,
        'tiled': False}

    pctcover = rasterize_pctcover(geom, atrans=atrans, shape=shape)
    with rasterio.open(output_raster_tif, 'w', **profile) as dst:
        dst.write(pctcover, 1)


print ("done")

done


C:\Users\ManuelLöhnertz\AppData\Local\Temp\ipykernel_22876\3542578645.py:68: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  geom = shape(next(src)['geometry'])  # take first feature's shapely geometry
C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\rasterio\__init__.py:331: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


# (2) shp to raster using gdal

In [27]:
from osgeo import gdal, ogr

# Define pixel_size and NoData value of new raster
pixel_size = 100
NoData_value = -9999
# Filename of input OGR file
vector_fn  = "N:/C2205_FAIRiCUBE/f02_data/tmp/test_vector_raster/vienna_polygon.shp"#
# Define the desired output raster shape (e.g., 1000x1000 pixels)
raster_fn = "N:/C2205_FAIRiCUBE/f02_data/tmp/test_vector_raster/vienna_polygon_100m_epsg3035" +str(shape[0])+"gdal.tif"



# Open the data source and read in the extent
source_ds = ogr.Open(vector_fn)
source_layer = source_ds.GetLayer()
x_min, x_max, y_min, y_max = source_layer.GetExtent()

# Create the destination data source
x_res = int((x_max - x_min) / pixel_size)
y_res = int((y_max - y_min) / pixel_size)
target_ds = gdal.GetDriverByName('GTiff').Create(raster_fn, x_res, y_res, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)

# Rasterize
gdal.RasterizeLayer(target_ds, [1], source_layer, burn_values=[0])

print ()

0

# (4) gdal script for reprojection the raster file

In [ ]:
from osgeo import gdal  # shifting with gdasl

input_ds = gdal.Open(r"N:\C2205_FAIRiCUBE\f02_data\d100_species_distribution\data\species_2490266_raster_10m.tif", gdal.GA_ReadOnly)

# (5) gdal script for shifting the raster to the correct loaction:


In [ ]:
from osgeo import gdal  # shifting with gdasl
shift_y = 10
input_ds = gdal.Open(r"N:\C2205_FAIRiCUBE\f02_data\d100_species_distribution\data\species_2490266_raster_10m.tif", gdal.GA_ReadOnly)
geo_transform = input_ds.GetGeoTransform()
new_geo_transform = list(geo_transform)
new_geo_transform[3] += shift_y
# Create a new dataset with the modified GeoTransform
driver = input_ds.GetDriver()

input_ds.SetGeoTransform(new_geo_transform)


output_ds = driver.CreateCopy(r"N:\C2205_FAIRiCUBE\f02_data\d100_species_distribution\data\species_2490266_raster_10m_gdal.tif", input_ds)


Show the results:

In [ ]:
# Load your raster file (replace 'your_raster.tif' with your actual file path)
raster_path = output_raster_tif
with rasterio.open(raster_path) as src:
    raster_array = src.read(1)  # Read the first band (adjust if needed)




plt.imshow(raster_array, cmap='coolwarm')  # You can choose a colormap (e.g., 'gray')
plt.colorbar()  # Add a colorbar if desired
plt.title("show the last written raster")
plt.show()